In [1]:
# Python
import os
import random

# Torch
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data.sampler import SubsetRandomSampler

# Torchvison
import torchvision.transforms as T

# Utils
from tqdm import tqdm
from utils import *

# Custom
from ptflops import get_model_complexity_info
import nets
import datasets as datasets
import methods as methods

# Seed
random.seed(0)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True

In [2]:
import easydict
 
args = easydict.EasyDict({
        # Setup
        "dataset": 'CIFAR10',
        "resolution": 32,
        "model":"ResNet18",
        "seed": 0,
        "data_path":'../data',
        "gpu":[0],
        "workers": 20,
        "print_freq":300,
        # AL
        "cycle": 10,
        "n_query": 1000,
        "subset": 50000,
        # Optimization
        "epochs":100,
        "batch_size": 64,
        "test_batch_size": 500,
        "optimizer": 'SGD',
        "lr":0.1,
        "min_lr": 1e-4,
        "momentum": 0.9,
        "weight_decay": 5e-4,
        "nesterov": True,
        "scheduler": "CosineAnnealingLR",
        "selection_optimizer": "SGD",
        "selection_batch": 128,
        "selection_momentum": 0.9,
        "selection_weight_decay": 5e-4,
        "selection_nesterov": True,
        # Coreset
        "method": "Uncertainty", #kCenterGreedy Uncertainty, GTknownUncertainty
        "balance": False,
        "uncertainty": 'LeastConfidence',
        "submodular": "GraphCut",
        "submodular_greedy": "LazyGreedy",
        "resume": 10
})
args.device = 'cuda' if torch.cuda.is_available() else 'cpu'


# 1. Seen, Uncertainty

In [5]:
n_train = 10000
Warm_up_epochs = 10

In [6]:
channel, im_size, num_classes, class_names, mean, std, dst_train, dst_u_all, dst_test = datasets.__dict__[args.dataset](args)
args.channel, args.im_size, args.num_classes, args.class_names = channel, im_size, num_classes, class_names
print("im_size: ", dst_train[0][0].shape)

# Initialize Unlabeled Set & Labeled Set
indices = list(range(len(dst_train)))

labeled_set = indices
unlabeled_set = indices[n_train:]

dst_subset = torch.utils.data.Subset(dst_train, labeled_set)
print("Initial set size: ", len(dst_subset))

# BackgroundGenerator for ImageNet to speed up dataloaders
if args.dataset == "ImageNet" or args.dataset == "ImageNet30":
    train_loader = DataLoaderX(dst_subset, batch_size=args.batch_size, shuffle=True, num_workers=args.workers, pin_memory=False)
    test_loader = DataLoaderX(dst_test, batch_size=args.test_batch_size, shuffle=False, num_workers=args.workers, pin_memory=False)
else:
    train_loader = torch.utils.data.DataLoader(dst_subset, batch_size=args.batch_size, shuffle=True, num_workers=args.workers, pin_memory=False)
    test_loader = torch.utils.data.DataLoader(dst_test, batch_size=args.test_batch_size, shuffle=False, num_workers=args.workers, pin_memory=False)

# Get Model
print("| Training on model %s" % args.model)
network = get_model(args, nets, args.model)

macs, params = get_model_complexity_info(network, (3, im_size[0], im_size[1]), as_strings=True, print_per_layer_stat=False, verbose=False)
print('{:<30}  {:<8}'.format('MACs: ', macs))
print('{:<30}  {:<8}'.format('Number of parameters: ', params))

# Get optim configurations for Distrubted SGD
criterion, optimizer, scheduler, rec = get_optim_configurations(args, network, train_loader)

im_size:  torch.Size([3, 32, 32])
Initial set size:  50000
| Training on model ResNet18
MACs:                           556.65 MMac
Number of parameters:           11.17 M 
lr: 0.1, momentum: 0.9, decay: 0.0005


In [7]:
# Training
print("==========Start Training==========")
for epoch in range(Warm_up_epochs):
    # train for one epoch
    train(train_loader, network, criterion, optimizer, scheduler, epoch, args, rec, if_weighted=False)

acc = test(test_loader, network, criterion, epoch, args, rec)
print('Label set size {}: Test acc {}'.format(len(labeled_set), acc))

==========Start Training==========
Test acc: * Prec@1 77.230
Label set size 50000: Test acc 77.23000144958496


In [8]:
dst_unlabeled = torch.utils.data.Subset(dst_u_all, unlabeled_set)
unlabeled_loader = torch.utils.data.DataLoader(dst_unlabeled, batch_size=args.test_batch_size, shuffle=False, num_workers=args.workers)

In [9]:
scores = np.array([])
batch_num = len(unlabeled_loader)

network.eval()
with torch.no_grad():
    for i, (input, _) in enumerate(unlabeled_loader):
        if i % args.print_freq == 0:
            print("| Selecting for batch [%3d/%3d]" % (i + 1, batch_num))
        if args.uncertainty == "LeastConfidence":
            scores = np.append(scores, network(input.to(args.device)).max(axis=1).values.cpu().numpy())
        elif args.uncertainty == "Entropy":
            preds = torch.nn.functional.softmax(network(input.to(args.device)), dim=1).cpu().numpy()
            scores = np.append(scores, (np.log(preds + 1e-6) * preds).sum(axis=1))
        elif args.uncertainty == 'Margin':
            preds = torch.nn.functional.softmax(network(input.to(args.device)), dim=1)
            preds_argmax = torch.argmax(preds, dim=1)
            max_preds = preds[torch.ones(preds.shape[0], dtype=bool), preds_argmax].clone()
            preds[torch.ones(preds.shape[0], dtype=bool), preds_argmax] = -1.0
            preds_sub_argmax = torch.argmax(preds, dim=1)
            scores = np.append(scores, (max_preds - preds[torch.ones(preds.shape[0], dtype=bool), preds_sub_argmax]).cpu().numpy())

uncertain_sorted_idx = np.argsort(scores)
print("# of uncertain_sorted_idx: ", len(uncertain_sorted_idx))

| Selecting for batch [  1/ 80]
# of uncertain_sorted_idx:  40000


In [10]:
labeled_set = indices[:n_train]

n_grid = 5
grid_size = int(len(uncertain_sorted_idx)/n_grid)
logs = []
for i in range(n_grid):
    selected_indices = uncertain_sorted_idx[i*grid_size:(i+1)*grid_size]
    adding_set = [unlabeled_set[idx] for idx in selected_indices]

    new_labeled_set = np.append(labeled_set, adding_set)
    assert len(new_labeled_set) == len(set(list(new_labeled_set)))

    dst_subset = torch.utils.data.Subset(dst_train, new_labeled_set)
    print("Grid: {},  Extended set size: {}".format(i, len(dst_subset)))

    # BackgroundGenerator for ImageNet to speed up dataloaders
    if args.dataset == "ImageNet" or args.dataset == "ImageNet30":
        train_loader = DataLoaderX(dst_subset, batch_size=args.batch_size, shuffle=True, num_workers=args.workers, pin_memory=False)
        test_loader = DataLoaderX(dst_test, batch_size=args.test_batch_size, shuffle=False, num_workers=args.workers, pin_memory=False)
    else:
        train_loader = torch.utils.data.DataLoader(dst_subset, batch_size=args.batch_size, shuffle=True, num_workers=args.workers, pin_memory=False)
        test_loader = torch.utils.data.DataLoader(dst_test, batch_size=args.test_batch_size, shuffle=False, num_workers=args.workers, pin_memory=False)
    
    # Get Model
    network = get_model(args, nets, args.model)
    criterion, optimizer, scheduler, rec = get_optim_configurations(args, network, train_loader)

    print("==========Start Training==========")
    for epoch in range(args.epochs):
        # train for one epoch
        train(train_loader, network, criterion, optimizer, scheduler, epoch, args, rec, if_weighted=False)

    acc = test(test_loader, network, criterion, epoch, args, rec)
    print('Grid {}: Test acc {}'.format(i, acc))
    logs.append([acc])
    print("==========Finished==========")
    

Grid: 0,  Extended set size: 18000
lr: 0.1, momentum: 0.9, decay: 0.0005
==========Start Training==========
Test acc: * Prec@1 92.230
Grid 0: Test acc 92.23000106811523
==========Finished==========
Grid: 1,  Extended set size: 18000
lr: 0.1, momentum: 0.9, decay: 0.0005
==========Start Training==========
Test acc: * Prec@1 91.570
Grid 1: Test acc 91.57000160217285
==========Finished==========
Grid: 2,  Extended set size: 18000
lr: 0.1, momentum: 0.9, decay: 0.0005
==========Start Training==========
Test acc: * Prec@1 90.230
Grid 2: Test acc 90.2300018310547
==========Finished==========
Grid: 3,  Extended set size: 18000
lr: 0.1, momentum: 0.9, decay: 0.0005
==========Start Training==========
Test acc: * Prec@1 88.920
Grid 3: Test acc 88.92000198364258
==========Finished==========
Grid: 4,  Extended set size: 18000
lr: 0.1, momentum: 0.9, decay: 0.0005
==========Start Training==========
Test acc: * Prec@1 88.350
Grid 4: Test acc 88.35000152587891
==========Finished==========


# Unseen, Uncertainty

In [11]:
channel, im_size, num_classes, class_names, mean, std, dst_train, dst_u_all, dst_test = datasets.__dict__[args.dataset](args)
args.channel, args.im_size, args.num_classes, args.class_names = channel, im_size, num_classes, class_names
print("im_size: ", dst_train[0][0].shape)

# Initialize Unlabeled Set & Labeled Set
indices = list(range(len(dst_train)))

labeled_set = indices[:n_train]
unlabeled_set = indices[n_train:]

dst_subset = torch.utils.data.Subset(dst_train, labeled_set)
print("Initial set size: ", len(dst_subset))

# BackgroundGenerator for ImageNet to speed up dataloaders
if args.dataset == "ImageNet" or args.dataset == "ImageNet30":
    train_loader = DataLoaderX(dst_subset, batch_size=args.batch_size, shuffle=True, num_workers=args.workers, pin_memory=False)
    test_loader = DataLoaderX(dst_test, batch_size=args.test_batch_size, shuffle=False, num_workers=args.workers, pin_memory=False)
else:
    train_loader = torch.utils.data.DataLoader(dst_subset, batch_size=args.batch_size, shuffle=True, num_workers=args.workers, pin_memory=False)
    test_loader = torch.utils.data.DataLoader(dst_test, batch_size=args.test_batch_size, shuffle=False, num_workers=args.workers, pin_memory=False)

# Get Model
print("| Training on model %s" % args.model)
network = get_model(args, nets, args.model)

macs, params = get_model_complexity_info(network, (3, im_size[0], im_size[1]), as_strings=True, print_per_layer_stat=False, verbose=False)
print('{:<30}  {:<8}'.format('MACs: ', macs))
print('{:<30}  {:<8}'.format('Number of parameters: ', params))

# Get optim configurations for Distrubted SGD
criterion, optimizer, scheduler, rec = get_optim_configurations(args, network, train_loader)

im_size:  torch.Size([3, 32, 32])
Initial set size:  10000
| Training on model ResNet18
MACs:                           556.65 MMac
Number of parameters:           11.17 M 
lr: 0.1, momentum: 0.9, decay: 0.0005


In [16]:
# Training
print("==========Start Training==========")
factor = int(len(dst_train)/n_train)
print(factor)
for epoch in range(factor*Warm_up_epochs):
    # train for one epoch
    train(train_loader, network, criterion, optimizer, scheduler, epoch, args, rec, if_weighted=False)

acc = test(test_loader, network, criterion, epoch, args, rec)
print('Label set size {}: Test acc {}'.format(len(labeled_set), acc))

==========Start Training==========
5
Test acc: * Prec@1 78.800
Label set size 10000: Test acc 78.80000114440918


In [17]:
dst_unlabeled = torch.utils.data.Subset(dst_u_all, unlabeled_set)
unlabeled_loader = torch.utils.data.DataLoader(dst_unlabeled, batch_size=args.test_batch_size, shuffle=False, num_workers=args.workers)

In [18]:
scores = np.array([])
batch_num = len(unlabeled_loader)

network.eval()
with torch.no_grad():
    for i, (input, _) in enumerate(unlabeled_loader):
        if i % args.print_freq == 0:
            print("| Selecting for batch [%3d/%3d]" % (i + 1, batch_num))
        if args.uncertainty == "LeastConfidence":
            scores = np.append(scores, network(input.to(args.device)).max(axis=1).values.cpu().numpy())
        elif args.uncertainty == "Entropy":
            preds = torch.nn.functional.softmax(network(input.to(args.device)), dim=1).cpu().numpy()
            scores = np.append(scores, (np.log(preds + 1e-6) * preds).sum(axis=1))
        elif args.uncertainty == 'Margin':
            preds = torch.nn.functional.softmax(network(input.to(args.device)), dim=1)
            preds_argmax = torch.argmax(preds, dim=1)
            max_preds = preds[torch.ones(preds.shape[0], dtype=bool), preds_argmax].clone()
            preds[torch.ones(preds.shape[0], dtype=bool), preds_argmax] = -1.0
            preds_sub_argmax = torch.argmax(preds, dim=1)
            scores = np.append(scores, (max_preds - preds[torch.ones(preds.shape[0], dtype=bool), preds_sub_argmax]).cpu().numpy())

uncertain_sorted_idx_unseen = np.argsort(scores)
print("# of uncertain_sorted_idx_unseen: ", len(uncertain_sorted_idx_unseen))

| Selecting for batch [  1/ 80]
# of uncertain_sorted_idx_unseen:  40000


In [19]:
labeled_set = indices[:n_train]

n_grid = 5
grid_size = int(len(uncertain_sorted_idx)/n_grid)
logs = []
for i in range(n_grid):
    selected_indices = uncertain_sorted_idx[i*grid_size:(i+1)*grid_size]
    adding_set = [unlabeled_set[idx] for idx in selected_indices]

    new_labeled_set = np.append(labeled_set, adding_set)
    assert len(new_labeled_set) == len(set(list(new_labeled_set)))

    dst_subset = torch.utils.data.Subset(dst_train, new_labeled_set)
    print("Grid: {},  Extended set size: {}".format(i, len(dst_subset)))

    # BackgroundGenerator for ImageNet to speed up dataloaders
    if args.dataset == "ImageNet" or args.dataset == "ImageNet30":
        train_loader = DataLoaderX(dst_subset, batch_size=args.batch_size, shuffle=True, num_workers=args.workers, pin_memory=False)
        test_loader = DataLoaderX(dst_test, batch_size=args.test_batch_size, shuffle=False, num_workers=args.workers, pin_memory=False)
    else:
        train_loader = torch.utils.data.DataLoader(dst_subset, batch_size=args.batch_size, shuffle=True, num_workers=args.workers, pin_memory=False)
        test_loader = torch.utils.data.DataLoader(dst_test, batch_size=args.test_batch_size, shuffle=False, num_workers=args.workers, pin_memory=False)
    
    # Get Model
    network = get_model(args, nets, args.model)
    criterion, optimizer, scheduler, rec = get_optim_configurations(args, network, train_loader)

    print("==========Start Training==========")
    for epoch in range(args.epochs):
        # train for one epoch
        train(train_loader, network, criterion, optimizer, scheduler, epoch, args, rec, if_weighted=False)

    acc = test(test_loader, network, criterion, epoch, args, rec)
    print('Grid {}: Test acc {}'.format(i, acc))
    logs.append([acc])
    print("==========Finished==========")
    

Grid: 0,  Extended set size: 18000
lr: 0.1, momentum: 0.9, decay: 0.0005
==========Start Training==========
Test acc: * Prec@1 91.590
Grid 0: Test acc 91.59000167846679
==========Finished==========
Grid: 1,  Extended set size: 18000
lr: 0.1, momentum: 0.9, decay: 0.0005
==========Start Training==========
Test acc: * Prec@1 91.100
Grid 1: Test acc 91.10000076293946
==========Finished==========
Grid: 2,  Extended set size: 18000
lr: 0.1, momentum: 0.9, decay: 0.0005
==========Start Training==========
Test acc: * Prec@1 90.460
Grid 2: Test acc 90.46000175476074
==========Finished==========
Grid: 3,  Extended set size: 18000
lr: 0.1, momentum: 0.9, decay: 0.0005
==========Start Training==========
Test acc: * Prec@1 89.210
Grid 3: Test acc 89.21000175476074
==========Finished==========
Grid: 4,  Extended set size: 18000
lr: 0.1, momentum: 0.9, decay: 0.0005
==========Start Training==========
Test acc: * Prec@1 88.180
Grid 4: Test acc 88.18000183105468
==========Finished==========
